# setup

In [1]:
#!g1.1
# %pip install transformers==4.27.1
# %pip install transformers[sentencepiece]

In [2]:
#!g1.1
!nvidia-smi

Fri Mar 17 09:30:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:8C:00.0 Off |                    0 |
| N/A   27C    P0    25W / 250W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [168]:
#!g1.1
!rm -rf .cache/

In [3]:
#!g1.1
%run -n main.py
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)

model_name = BLOOM_RU
cache_dir = '.cache'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

In [4]:
#!g1.1
tokenizer.pad_token = '<pad>'
tokenizer.padding_side = 'left'

In [5]:
#!g1.1
from transformers import GenerationConfig

generation_config = GenerationConfig(
    do_sample=True,
    max_new_tokens=32,
    pad_token_id=tokenizer.pad_token_id
)

In [6]:
#!g1.1
prompt = '''
Однажды
'''
output = generator(
    prompt,
    generation_config=generation_config,
    return_full_text=False,
    num_return_sequences=4
)
output

[{'generated_text': 'В одной из улиц Саратова около восьми часов ночи сгорела продажа. О'},
 {'generated_text': 'Обучением плаванию, тренировками физической подготовки. Трениров'},
 {'generated_text': 'световые специалисты на основании обратных гласит, что и как и с гряз'},
 {'generated_text': 'Как образование может влиять на психику человека? Казахстанские ученые у'}]

# eval

In [7]:
#!g1.1
%run -n main.py
task = TERRA
count = len(list(Path('evals').glob('*.jsonl')))
eval = f'{count + 1}_bloom_ru_{task}'
eval

'66_bloom_ru_terra'

In [8]:
#!g1.1
%run -n main.py
lines = read_lines(f'tasks/{task}.jsonl')
task_items = list(parse_jsonl(lines))

In [9]:
#!g1.1
%run -n main.py
prompts = [
    TASK_PROMPTS[task](_)
    for _ in task_items
]
prompt = random.choice(prompts)
print(prompt)

Прочитай текст, проверь верно ли утверждение.
Ответь коротко: да или нет. Если не уверен, выбери наиболее вероятный ответ.
---
Текст: Трижды он был привлечён судебным приставом к административной ответственности по ст. 17.15 КоАП РФ за неисполнение содержащихся в исполнительном документе требований неимущественного характера. Так как срок для добровольного исполнения истёк, пристрой снесли принудительно.
Утверждение: Пристрой был снесен.
Верно: Да
---
Текст: Для молодого организма это не прошло бесследно. Резкое токсическое воздействие этанола привело к смерти парня. Его тело обнаружила бабушка, которая вернулась на следующий день.
Утверждение: Молодой организм стал сильнее от этанола.
Верно: Нет
---
Текст: Я давно хотела слепить снежный мотоцикл, но вот мы только собрались, примерились к этой куче, а я глянула и поняла, что вижу там только Будду. Доброго такого, пузатого, довольного и умиротворенного, - рассказывает Дарья. - Пока делали, несколько раз приходилось объяснять соседям, чт

In [10]:
#!g1.1
generator(
    prompt,
    generation_config=generation_config,
    return_full_text=False,
    num_return_sequences=4
)

[{'generated_text': 'рылеет стройная шар. Но спровадили человека, охватил всех и тут'},
 {'generated_text': 'рыбак не только привлекает внимание взрослых, но по-прежнему держит в'},
 {'generated_text': 'юноша, я знаю, но прожил ее совсем не мало, поэтому Будда и с'},
 {'generated_text': 'рыжей снежной брат малон, а может, и это что пошло с добрыми друзь'}]

In [12]:
#!g1.1
output = generator(
    prompts,
    generation_config=generation_config,
    return_full_text=False,
    batch_size=4
)

In [13]:
#!g1.1
random.sample(output, 3)

[[{'generated_text': 'юридически не вызывает сомнения тот факт, что акционерное общество я'}],
 [{'generated_text': 'риск ведь есть определённый. Например, на территории Крыма есть о'}],
 [{'generated_text': 'рыбка, она чем остается заячьим защитником? Если сыпать зубряку'}]]

In [14]:
#!g1.1
eval_items = []
for task_item, response_item in zip(task_items, output):
    response = response_item[0]['generated_text']

    eval_items.append({
        'id': task_item['id'],
        'response': response
    })

In [15]:
#!g1.1
%run -n main.py
id_targets = {
    _['id']: _['label']
    for _ in task_items
}
norm_response = NORM_RESPONSES[task]
id_preds = {
    _['id']: norm_response(_['response'])
    for _ in eval_items
}
acc_score(id_targets, id_preds)

(0.4090909090909091, 78)

In [16]:
#!g1.1
lines = format_jsonl(eval_items)
write_lines(f'evals/{eval}.jsonl', lines)

In [17]:
#!g1.1
for task in TASKS[1:]:
    count = len(list(Path('evals').glob('*.jsonl')))
    eval = f'{count + 1}_bloom_ru_{task}'

    lines = read_lines(f'tasks/{task}.jsonl')
    task_items = list(parse_jsonl(lines))
    
    prompts = [
        TASK_PROMPTS[task](_)
        for _ in task_items
    ]
    
    output = generator(
        prompts,
        generation_config=generation_config,
        return_full_text=False,
        batch_size=4
    )
    
    eval_items = []
    for task_item, response_item in zip(task_items, output):
        response = response_item[0]['generated_text']
        eval_items.append({
            'id': task_item['id'],
            'response': response
        })
    
    lines = format_jsonl(eval_items)
    write_lines(f'evals/{eval}.jsonl', lines)

In [ ]:
#!g1.1
